<a href="https://colab.research.google.com/github/zizoupavon/First/blob/main/How_to_Interact_with_APIs_Using_Function_Calling_in_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
pip install --upgrade google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.43.0
    Uninstalling google-auth-2.43.0:
      Successfully uninstalled google-auth-2.43.0
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.130.0
    Uninstalling google-cloud-aiplatform-1.130.0:
      Successfully uninstalled google-cloud-aiplatform-1.130.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.45.0 which is incompatible.


In [3]:
import vertexai
from vertexai.generative_models import GenerativeModel

# Initialize Vertex AI with your project ID and region
vertexai.init(project="gen-lang-client-0131371905", location="us-central1")

model = GenerativeModel("gemini-2.0-flash-lite-001")

/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [4]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

Unfortunately, I cannot give you a real-time exchange rate. These rates fluctuate constantly.

However, here's how you can find the current exchange rate:

*   **Use a reliable financial website:**
    *   Google Finance: Simply search "EUR to USD" on Google.
    *   Yahoo Finance: Go to finance.yahoo.com and search "EURUSD=X"
    *   Major financial news websites: Websites like Bloomberg, Reuters, and The Wall Street Journal have up-to-the-minute exchange rates.
*   **Use a currency converter app:** There are many free currency converter apps available for your phone.
*   **Check with your bank or a currency exchange service:** They will provide the most up-to-date rate, but keep in mind they may include a markup or fees.

Remember to double-check the source to make sure you are getting the most recent information.



In [5]:
user_prompt = "What's the exchange rate from euros to US dollars today?"

response = model.generate_content("""
Your task is to extract parameters from the user's input and return it as a
structured JSON payload. The user will ask about the exchange rate and which
currency they are converting from and converting to.

User input: {user_prompt}

Please extract the currencies as parameters and put them in a JSON object.
""".format(user_prompt=user_prompt))
print(response.text)

```json
{
  "from_currency": "euros",
  "to_currency": "US dollars"
}
```


In [6]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
response.text

'{"amount":1.0,"base":"EUR","date":"2025-12-24","rates":{"AUD":1.7587,"BGN":1.9558,"BRL":6.5076,"CAD":1.6128,"CHF":0.9284,"CNY":8.2679,"CZK":24.271,"DKK":7.4694,"GBP":0.8729,"HKD":9.1655,"HUF":389.53,"IDR":19740,"ILS":3.7514,"INR":105.78,"ISK":148.0,"JPY":183.83,"KRW":1707.04,"MXN":21.102,"MYR":4.7678,"NOK":11.804,"NZD":2.0197,"PHP":69.269,"PLN":4.2135,"RON":5.0899,"SEK":10.8055,"SGD":1.5135,"THB":36.593,"TRY":50.507,"USD":1.1787,"ZAR":19.6091}}'

In [7]:
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-2.0-flash-lite-001")

In [8]:
openapi_schema = """
openapi: 3.0.0
info:
  title: Frankfurter Exchange Rate API
  description: This API provides current and historical exchange rates
  version: 1.0.0
servers:
  - url: https://api.frankfurter.app
paths:
  /{date}:
    get:
      summary: Get the latest currency exchange rates.
      parameters:
        - name: date
          in: path
          description: Get currency rates for a specific date or 'latest' if a date is not specified
          required: true
          schema:
            type: string
        - name: from
          in: query
          description: The currency to convert from.
          required: true
          schema:
            type: string
        - name: to
          in: query
          description: The currency to convert to.
          schema:
            type: string
"""

In [9]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

In [10]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

In [16]:
prompt = """What is the exchange rate from Indian Rupee to Canadian dollars?
How much is 4000000 Indian Rupees worth in Canadian dollars on dec 26, 2025?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_to"
        value {
          string_value: "CAD"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "INR"
        }
      }
      fields {
        key: "currency_date"
        value {
          string_value: "2025-12-26"
        }
      }
    }
  }
}



In [17]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'from': 'INR', 'date': '2025-12-26', 'to': 'CAD'}

In [18]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

'{"amount":1.0,"base":"INR","date":"2025-12-24","rates":{"CAD":0.01525}}'

In [20]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date. End with a funny joke about the name Kuljit"""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

"Here's how to calculate that:\n\n1.  **Find the exchange rate:** The exchange rate from Indian Rupees (INR) to Canadian Dollars (CAD) on December 26, 2025, is 0.01525. This means that 1 Indian Rupee is worth 0.01525 Canadian Dollars. (The API provides the rate for Dec 24, 2025, I am using this as a close approximation for Dec 26, 2025)\n\n2.  **Calculate the value in CAD:** To find out how much 4,000,000 Indian Rupees is worth in Canadian Dollars, multiply the amount in INR by the exchange rate: 4,000,000 INR \\* 0.01525 CAD/INR = 61,000 CAD\n\nSo, 4,000,000 Indian Rupees is worth 61,000 Canadian Dollars on December 26, 2025.\n\nWhy did Kuljit bring a ladder to the library? Because he heard the books were stacked up!\n"